In [1]:
from enum import Enum
from cuery import Prompt, ResponseModel, Task
from pydantic import Field

# Choices (enum)
Require LLM to respond with one of N _options_ (fixed categories).

In [2]:
class Role(Enum):
    PRINCIPAL = "PRINCIPAL"
    TEACHER = "TEACHER"
    STUDENT = "STUDENT"
    OTHER = "OTHER"


class UserDetail(ResponseModel):
    age: int
    name: str
    role: Role = Field(description="Correctly assign one of the predefined roles to the user.")


prompt = Prompt(
    messages=[
        {
            "role": "user",
            "content": "Please a create a synthetic user profile with age, name and role.",
        }
    ]
)

task = Task("user_profile", prompt=prompt, response=UserDetail)

response = await task()
print(response)
response.to_pandas()

[UserDetail(age=25, name='John Doe', role=<Role.STUDENT: 'STUDENT'>)]


,age,name,role
0,25,John Doe,Role.STUDENT


# Simple Multivalued fields
Require LLM to respond with a _list_ of values (unconstrained).

In [3]:
class Ingredients(ResponseModel):
    items: list[str] = Field(description="List of ingredients for the recipe.")


prompt = Prompt(
    messages=[
        {
            "role": "user",
            "content": "List the ingredients for the following dish: {{dish}}.",
        }
    ],
    required=["dish"],
)

context = [{"dish": "pasta bolognese"}, {"dish": "chocolate cake"}]
task = Task("dishes", prompt=prompt, response=Ingredients)
responses = await task(context=context)
print(responses)

Iterating context:   0%|          | 0/2 [00:00<?, ?it/s]

[Ingredients(items=['pasta', 'bolognese sauce', 'ground beef', 'onion', 'garlic', 'carrot', 'celery', 'red wine', 'tomato paste', 'beef broth', 'salt', 'pepper', 'olive oil', 'parmesan cheese']), Ingredients(items=['flour', 'cocoa powder', 'sugar', 'eggs', 'butter', 'baking soda', 'baking powder', 'salt', 'vanilla extract', 'milk', 'hot water'])]


In [4]:
responses.to_pandas(explode=False)

,dish,items
0,pasta bolognese,"[pasta, bolognese sauce, ground beef, onion, g..."
1,chocolate cake,"[flour, cocoa powder, sugar, eggs, butter, bak..."


In [5]:
responses.to_records(explode=False)

[{'dish': 'pasta bolognese',
  'items': ['pasta',
   'bolognese sauce',
   'ground beef',
   'onion',
   'garlic',
   'carrot',
   'celery',
   'red wine',
   'tomato paste',
   'beef broth',
   'salt',
   'pepper',
   'olive oil',
   'parmesan cheese']},
 {'dish': 'chocolate cake',
  'items': ['flour',
   'cocoa powder',
   'sugar',
   'eggs',
   'butter',
   'baking soda',
   'baking powder',
   'salt',
   'vanilla extract',
   'milk',
   'hot water']}]